In [2]:
pip install -qU langchain langchain-openai langchain-anthropic langchain-community wikipedia langchain-pinecone

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import openai
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
import os 
import getpass

_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

llm = ChatOpenAI(model = "gpt-3.5-turbo")
os.environ['USER_AGENT'] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"

In [12]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

system_prompt = (
    "You are an expert extraction algorithm. "
    "Only extract relevant information from the text. "
    "If you do not know the value of an attribute asked "
    "to extract, return null for the attribute's value. "
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("examples"),
        ("human","{text}")
    ]
)

In [13]:
from langchain_core.messages import HumanMessage

prompt.invoke({"text" : "this is some text","examples" : [HumanMessage(content = "testing 1 2 3")]})

ChatPromptValue(messages=[SystemMessage(content="You are an expert extraction algorithm. Only extract relevant information from the text. If you do not know the value of an attribute asked to extract, return null for the attribute's value. "), HumanMessage(content='testing 1 2 3'), HumanMessage(content='this is some text')])

In [14]:
from typing import List,Optional

from langchain_core.pydantic_v1 import BaseModel,Field
from langchain_openai import ChatOpenAI

class Person(BaseModel):
    """Information about a person"""
    name : Optional[str] = Field(..., description = "The name of the person")
    hair_color: Optional[str] = Field(
        ...,
        description = "The color of the person's if known"
    )
    height_in_meters:Optional[str] = Field(
        ...,
        description = "Height in METERs"
    )

class Data(BaseModel):
    """Extracted data about people. """
    people : List[Person]

In [7]:
import uuid
from typing import Dict,List,TypedDict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)
from langchain_core.pydantic_v1 import BaseModel, Field

class Example(TypedDict):
    input : str
    tool_calls : List[BaseModel]

def tool_example_to_messages(example: Example) -> List[BaseMessage]:
    messages: List[BaseMessage] = [HumanMessage(content = example["input"])]
    tool_calls = []
    
    for tool_call in example["tool_calls"]:
        tool_calls.append(
            {
                "id" : str(uuid.uuid4()),
                "args" : tool_call.dict(),
                "name" : tool_call.__class__.__name__
            }
        )
    messages.append(AIMessage(content = "",tool_calls = tool_calls))
    tool_outputs = example.get("tool_outputs") or [
        "You have correctly called this tool"
    ] * len(tool_calls)
    
    for output,tool_call in zip(tool_outputs,tool_calls):
        messages.append(ToolMessage(content = output,tool_call_id = tool_call["id"]))
    return messages

In [15]:
examples = [
    (
        "This ocean is vast and blue. Its more than 20,000 feet deep. There are many fish in it.",
        Person(name = None, height_in_meters = None, hair_color = None),
    ),
    (
        "Virat scored at an average of 10.24 in this edition's World Cup.",
        Person(name = "Virat", height_in_meters = None, hair_color = None),
    )
]

messages = []

for text,tool_call in examples:
    messages.extend(
        tool_example_to_messages({"input" : text,"tool_calls" : [tool_call]})
    )

In [17]:
example_prompt = prompt.invoke({"text" : "this is some text","examples" : messages})

for message in example_prompt.messages :
    print(f"{message.type} : {message}")
    print("\n\n")

system : content="You are an expert extraction algorithm. Only extract relevant information from the text. If you do not know the value of an attribute asked to extract, return null for the attribute's value. "



human : content='This ocean is vast and blue. Its more than 20,000 feet deep. There are many fish in it.'



ai : content='' tool_calls=[{'name': 'Person', 'args': {'name': None, 'hair_color': None, 'height_in_meters': None}, 'id': '86ed6ba5-f029-4954-8d4c-5b61ab470e5b'}]



tool : content='You have correctly called this tool' tool_call_id='86ed6ba5-f029-4954-8d4c-5b61ab470e5b'



human : content="Virat scored at an average of 10.24 in this edition's World Cup."



ai : content='' tool_calls=[{'name': 'Person', 'args': {'name': 'Virat', 'hair_color': None, 'height_in_meters': None}, 'id': 'f68875a1-f004-47f6-8d5f-90cf90f8c84a'}]



tool : content='You have correctly called this tool' tool_call_id='f68875a1-f004-47f6-8d5f-90cf90f8c84a'



human : content='this is some text'


